### get gene embeddings for the 500 HVG + SL dataset

In [3]:
# Imports

import sys
import pickle as pkl

#change to path to your Geneformer directory
sys.path.append('/work/magroup/kaileyhu/Geneformer')
from geneformer import EmbExtractor
from datasets import load_from_disk
import pandas as pd

from sklearn.decomposition import PCA

print("imports done")

imports done


In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

In [2]:
embex = EmbExtractor(model_type="Pretrained",
                     emb_mode="gene",
                     num_classes=0,
                     emb_layer=-1,
                     forward_batch_size=16,
                     nproc=16)

In [3]:
embs = embex.extract_embs("/work/magroup/kaileyhu/Geneformer/gf-12L-30M-i2048/",
                          "/work/magroup/kaileyhu/res/hvg_500_w_SL_tokenized_2048.dataset", # "/work/magroup/kaileyhu/res/hvg_500_tokenized_2048_no_metadata.dataset",
                          "/work/magroup/kaileyhu/res/gene_embeddings/",
                          "hvg_500_w_SL_tokenized_GENES" #"hvg_500_tokenized_2048_GENES"
                         )

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/home/kaileyhu/anaconda3/envs/geneformer/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is N

  0%|          | 0/63 [00:00<?, ?it/s]

In [5]:
ds = load_from_disk("/work/magroup/kaileyhu/res/hvg_500_w_SL_tokenized_2048.dataset")

In [6]:
all_genes = set()
for a in ds['input_ids']:
    for b in a:
        all_genes.add(b)

In [7]:
len(all_genes)

5421

In [13]:
TOKEN_DICTIONARY_FILE = "/work/magroup/kaileyhu/Geneformer/geneformer/token_dictionary_gc95M.pkl"

In [14]:
with open(TOKEN_DICTIONARY_FILE, "rb") as td:
    gene_token_dict = pickle.load(td)

In [15]:
gene_token_dict['ENSG00000012048']

312

### PCA them

In [8]:
gene_embs = pd.read_csv("/work/magroup/kaileyhu/res/gene_embeddings/hvg_500_w_SL_tokenized_GENES.csv")
gene_embs.set_index("Unnamed: 0", inplace = True)

In [9]:
pca = PCA(n_components=256)

principalComponents = pca.fit_transform(gene_embs)

In [10]:
pca_df = pd.DataFrame(data = principalComponents, columns = [f"pc {i}" for i in range(256)])

In [12]:
pca_df.index = gene_embs.index

In [14]:
pca_df.to_csv("/work/magroup/kaileyhu/res/gene_embeddings/HVG_SL_pca_256.csv")

In [4]:
with open ("/work/magroup/shared/Heimdall/data/pretrained_embeddings/gene2vec/gene2vec_genes.pkl", "rb") as f:
    temp = pkl.load(f)

In [6]:
len(temp['ENSG00000280109'])

200